In [94]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import joblib

In [95]:
tr_tr_new = joblib.load('../joblib/tr_tr_new.joblib')

In [96]:
te_tr_new = joblib.load('../joblib/te_tr_new.joblib')

In [97]:
import numpy as np
from catboost import CatBoostClassifier, Pool

In [98]:
# Identify categorical features. CatBoost can handle them natively.
cat_var = ['ProductCD',
           'card6', 'card2', 'card5', 'card1', 'card4', 'card3',
           'addr2', 'addr1',
           'P_emaildomain',
           'R_emaildomain',
           'M6', 'M7', 'M8', 'M4', 'M9', 'M1', 'M2', 'M5', 'M3',
           'DeviceType',
           'DeviceInfo',
           'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19',
           'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27',
           'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35',
           'id_36', 'id_37', 'id_38']

In [99]:
te_tr_new.rename(columns=lambda col: col.replace('id-', 'id_'), inplace=True)

In [100]:
for col in cat_var:
    tr_tr_new[col] = tr_tr_new[col].astype(str)
    te_tr_new[col] = te_tr_new[col].astype(str)

In [101]:
from sklearn.model_selection import train_test_split
X = tr_tr_new.drop('isFraud', axis=1)
y = tr_tr_new['isFraud']

In [102]:
# Split the data into training and temp sets (80% train, 20% temp)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)

# Split the temp data into validation and test sets (50% validation, 50% test)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(X_test)}")

Training set size: 472432
Validation set size: 59054
Test set size: 59054


In [66]:
# Initialize CatBoostClassifier
model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.07,
    depth=7,
    cat_features=cat_var,
    eval_metric='AUC',  # Changed metric to AUC
    random_seed=42,
    loss_function='Logloss',
    metric_period=500,
    task_type='GPU'
)

In [68]:
import warnings

warnings.filterwarnings('ignore')
model.fit(
    X_train, y_train,
    cat_features=cat_var,
    eval_set=(X_val, y_val),
    logging_level='Verbose',  # Output to stdout
    plot=True  # Plotting training and validation error (if you run this notebook locally)
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.7390227	best: 0.7390227 (0)	total: 298ms	remaining: 2m 28s
499:	test: 0.9583119	best: 0.9583119 (499)	total: 2m 3s	remaining: 0us
bestTest = 0.9583119154
bestIteration = 499


In [69]:
from sklearn.metrics import roc_auc_score

# Predict probabilities for the positive class
preds_proba = model.predict_proba(X_test)[:, 1]

# Evaluate model using AUC
auc_score = roc_auc_score(y_test, preds_proba)
print("AUC:", auc_score)

AUC: 0.9628169734061687


In [72]:
pred_te_tr_new_p = model.predict_proba(te_tr_new)[:, 1]

# Create the output DataFrame
output_df = pd.DataFrame({
    'TransactionID': te_tr_new.reset_index()['TransactionID'],
    'isFraud': pred_te_tr_new_p
})

# Save the DataFrame to a CSV file
output_df.to_csv('predicted_fraud_catboost.csv', index=False)


In [74]:
joblib.dump(model, '../models_libjob/cb[0.9316][0.9087].joblib')

['../models_libjob/cb[0.9316][0.9087].joblib']

In [116]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import itertools

params_grid = {
    'iterations': [5000],
    'learning_rate': [0.07],
    'depth': [7,8],
    'loss_function': ['Logloss'],
    'metric_period': [500]
}

# Convert all categorical features to string type
for col in cat_var:
    X[col] = X[col].astype(str)

# Initialize a DataFrame to store results
cv_results = pd.DataFrame(columns=['iterations', 'learning_rate', 'depth','loss_function', 'metric_period','mean_auc'])

kf = KFold(n_splits=5, shuffle=True, random_state=42)
num_iter = 0

for params in itertools.product(*params_grid.values()):
    # Unpack the current parameter set
    iterations, learning_rate, depth, loss_function, metric_period = params
    
    aucs = []
    for train_index, val_index in kf.split(X):
        X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
        y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]
        
        model = CatBoostClassifier(
            iterations=iterations,
            learning_rate=learning_rate,
            depth=depth,
            eval_metric='AUC',
            task_type='GPU',
            cat_features=cat_var,
            random_seed=42,
            verbose=0
        )
        
        model.fit(X_train_fold, y_train_fold)
        preds_proba = model.predict_proba(X_val_fold)[:, 1]
        fold_auc = roc_auc_score(y_val_fold, preds_proba)
        aucs.append(fold_auc)
    
    mean_auc = sum(aucs) / len(aucs)
    cv_results.loc[num_iter] = [iterations, learning_rate, depth, loss_function, metric_period, mean_auc]
    num_iter += 1

print(cv_results)


Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU


   iterations  learning_rate  depth loss_function  metric_period  mean_auc
0        5000           0.07      7       Logloss            500  0.975606
1        5000           0.07      8       Logloss            500  0.976749


In [117]:
best_params = cv_results.sort_values(by='mean_auc', ascending=False).iloc[0]

print(best_params)

iterations           5000
learning_rate        0.07
depth                   8
loss_function     Logloss
metric_period         500
mean_auc         0.976749
Name: 1, dtype: object


In [110]:
best_params['depth']

8

In [118]:
best_model = CatBoostClassifier(
    iterations=int(best_params['iterations']),
    learning_rate=float(best_params['learning_rate']),
    depth=int(best_params['depth']),
    cat_features=cat_var,
    eval_metric='AUC',  # Changed metric to AUC
    random_seed=42,
    loss_function=best_params['loss_function'],
    metric_period=int(best_params['metric_period']),
    task_type='GPU'
)

In [119]:
best_model.fit(
    X_train, y_train,
    cat_features=cat_var,
    eval_set=(X_val, y_val),
    logging_level='Verbose',  # Output to stdout
    plot=True  # Plotting training and validation error (if you run this notebook locally)
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.8016908	best: 0.8016908 (0)	total: 340ms	remaining: 28m 17s
500:	test: 0.9606386	best: 0.9606386 (500)	total: 2m 22s	remaining: 21m 21s
1000:	test: 0.9651848	best: 0.9651848 (1000)	total: 4m 43s	remaining: 18m 53s
1500:	test: 0.9678751	best: 0.9678751 (1500)	total: 7m 3s	remaining: 16m 27s
2000:	test: 0.9692303	best: 0.9692303 (2000)	total: 9m 24s	remaining: 14m 5s
2500:	test: 0.9706382	best: 0.9706382 (2500)	total: 11m 45s	remaining: 11m 45s
3000:	test: 0.9713688	best: 0.9713688 (3000)	total: 14m 6s	remaining: 9m 23s
3500:	test: 0.9722379	best: 0.9722379 (3500)	total: 16m 27s	remaining: 7m 2s
4000:	test: 0.9727167	best: 0.9727167 (4000)	total: 18m 47s	remaining: 4m 41s
4500:	test: 0.9732828	best: 0.9732828 (4500)	total: 21m 8s	remaining: 2m 20s
4999:	test: 0.9736134	best: 0.9736134 (4999)	total: 23m 30s	remaining: 0us
bestTest = 0.973613441
bestIteration = 4999


In [120]:
pred_te_tr_new_p = best_model.predict_proba(te_tr_new)[:, 1]

# Create the output DataFrame
output_df = pd.DataFrame({
    'TransactionID': te_tr_new.reset_index()['TransactionID'],
    'isFraud': pred_te_tr_new_p
})

# Save the DataFrame to a CSV file
output_df.to_csv('predicted_fraud_catboost[3].csv', index=False)


In [ ]:
joblib.dump(best_model, '../models_libjob/cb[0.9316][0.9087].joblib')